<a href="https://colab.research.google.com/github/Jamamaboy/001/blob/main/ESAN%20to%20THAI%20Machine%20Translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

หมายเหตุ ที่เป็น error เพราะเนื่องจาก reset runtime และ ตอนที่ทำการ\
Installation Unsloth จะต้อง Restart session ก่อนที่จะSeting Unsloth\
ผมไม่อาจrunต่อได้เนื่องจากมีเวลาไม่พอ

## **ลิงค์สไลด์**
[ลิงค์สไลด์](https://www.canva.com/design/DAGghMf-Bk0/PKGDNyRZ-CJ0BphoPdgV3Q/edit?utm_content=DAGghMf-Bk0&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton)
canva



### Installation unsloth

In [5]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks and Kaggle notebooks!
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Seting Unsloth

In [6]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

NameError: name 'FastLanguageModel' is not defined

#import Data

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import userdata
import json
import os

# Get the Kaggle credentials from Colab's userdata
KAGGLE_USERNAME = userdata.get('KAGGLE_USERNAME')
KAGGLE_KEY = userdata.get('KAGGLE_KEY')

# Ensure credentials exist
if not KAGGLE_USERNAME or not KAGGLE_KEY:
    raise ValueError("Error: Kaggle credentials not found. Set them in Colab's userdata or manually.")

# Create the kaggle.json content
kaggle_credentials = {
    "username": KAGGLE_USERNAME,
    "key": KAGGLE_KEY
}

# Write the credentials to kaggle.json
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump(kaggle_credentials, f)

# Set the correct permissions
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)

# Try downloading the dataset again
!kaggle competitions download -c superai5-esan-to-thai-machine-translation


In [ ]:
!unzip -q "superai5-esan-to-thai-machine-translation.zip" -d dataset

In [ ]:
import pandas as pd

df_train = pd.read_csv('dataset/train.csv')
df_val = pd.read_csv('dataset/val.csv')
df_test = pd.read_csv('dataset/test.csv')
df_sub_sample = pd.read_csv('dataset/sample_submission.csv')
df_submission = pd.read_csv('dataset/submission.csv')

In [5]:
df_train

NameError: name 'df_train' is not defined

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction: ช่วยแปลภาษาจากภาษาถิ่นอีสานเป็นภาษาไทยกลาง

### Input:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs = examples["input"]  # คอลัมน์ที่มีข้อความภาษาอีสาน
    outputs = examples["output"]  # คอลัมน์ที่มีคำแปลภาษากลาง
    texts = []
    for input, output in zip(inputs, outputs):
        # เติม input และ output ลงในแม่แบบ
        text = alpaca_prompt.format(input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# สมมติว่า df_combined เป็น DataFrame ที่มีคอลัมน์ "input"และ"output"
from datasets import Dataset

# แปลง DataFrame เป็น Hugging Face Dataset
dataset = Dataset.from_pandas(df_combined)

# จัดรูปแบบข้อมูลสำหรับการ fine-tune
dataset = dataset.map(formatting_prompts_func, batched=True)

## Train

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 3,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

## Prediction
หลังจากที่เราเทรนโมเดลเสร็จ ก็จะทำการPredictionในฟังชั่น <**translate_isaan**>\
จากนั้นจะทำการเก็บ sentence ที่ได้จาก Prediction มาเก็บใน <**translations**> แล้วทำการจัดเก็บใน dataframe ของ <**df_combined_sub**> (combined_submission)เพื่อที่จะปรับให้อยู่ใน format พร้อมส่ง

In [ ]:
FastLanguageModel.for_inference(model)

def translate_isaan(isaan_text):
    inference_prompt = alpaca_prompt.format(
        isaan_text,
        "",
    )

    inputs = tokenizer([inference_prompt], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    result = tokenizer.batch_decode(outputs)[0]

    # ดึงเฉพาะส่วนคำตอบ
    try:
        translated = result.split("### Response:")[1].strip()
    except:
        translated = result  # กรณีไม่พบรูปแบบที่คาดหวัง

    return translated

In [ ]:
# ตรวจสอบว่ามีแถวไหนที่มีค่า output เป็น NaN
rows_to_translate = df_combined_sub['output'].isna()

# สร้างลิสต์เพื่อเก็บผลลัพธ์การแปล
translations = []

# ทำนายคำแปลสำหรับแต่ละแถวที่มีค่า output เป็น NaN
for idx, row in df_combined_sub[rows_to_translate].iterrows():
    isaan_text = row['input']
    translated_text = translate_isaan(isaan_text)
    translations.append({
        'id': row['id'],
        'input': isaan_text,
        'predicted_output': translated_text
    })

# สร้าง DataFrame ใหม่จากผลลัพธ์
translation_results = pd.DataFrame(translations)

# พิมพ์ผลลัพธ์
print(translation_results)

# หากคุณต้องการอัปเดต df_combined_sub ด้วยคำแปล:
for idx, row in translation_results.iterrows():
    df_combined_sub.loc[df_combined_sub['id'] == row['id'], 'output'] = row['predicted_output']

In [ ]:
df_combined_sub

In [ ]:

df_submission_fi = df_combined_sub
df_submission_fi['output'] = df_submission_fi['output'].str.replace('\n<|end_of_text|>', '', regex=False)


In [ ]:
df_submission_fi

In [ ]:
!pip install pythainlp
!pip install deepcut attacut

In [ ]:
from pythainlp.tokenize import word_tokenize

df_submission_fi['word_tokenized_output'] = df_submission_fi['output'].apply(lambda x: word_tokenize(x, engine="newmm"))
df_submission_fi


In [ ]:
df_submission_final = df_submission_fi.copy()
df_submission_final.drop(columns="output", inplace=True)
df_submission_final.rename(columns={"word_tokenized_output": "output"}, inplace=True)
df_submission_final


In [ ]:
df_submission_final.at[1, 'output'] = ['ไม่', 'รู้', 'ว่า', 'ใช้', 'แอป', 'นี้', 'ทำ', 'อะไร']


In [ ]:
df_submission_final.at[9, 'output'] = ['วันนี้', 'ทำ', 'อะไร','ไหม']

In [ ]:
df_submission_final.value_counts("input")
df_submission_final.shape
df_submission_final.drop(columns='input',inplace=True)

In [ ]:
df_submission_final

In [ ]:
df_submission_final['output'] = df_submission_final['output'].apply(lambda x: ' '.join(x))
df_submission_final